In [4]:
# 初始化
import django_setup

In [5]:
from apps.projects.models import Project

In [ ]:
current_project = Project.objects.get(project_name='测试项目3')
print(f"用于测试的项目: {current_project.project_name}")
print(f"项目包含的文件: {current_project.files.all()}")

In [ ]:
from apps.projects.models import Task, TaskType
task = Task.objects.get(stage__project=current_project, type=TaskType.DOCX_EXTRACTION_TASK)
print(task.docx_tiptap)

In [ ]:
from django.core.exceptions import ValidationError
import os, tempfile, requests, uuid
import logging
logger = logging.getLogger(__name__)

# 获取文件的预签名URL
presigned_url = current_project.files.first().get_presigned_url()
if not presigned_url:
    raise ValidationError("无法获取文件访问地址")

logger.info(f"开始下载文件: project_id={current_project.id}, file={current_project.files.first().name}")

# 下载文件到临时文件
temp_file_path = os.path.join(tempfile.gettempdir(), f"doc_analysis_{uuid.uuid4()}.docx")
response = requests.get(presigned_url)
response.raise_for_status()

with open(temp_file_path, 'wb') as temp_file:
    temp_file.write(response.content)

In [ ]:
print(temp_file_path)

In [ ]:
from apps.projects.tiptap.mammoth import DocxToHtmlConverter

# 初始化转换器
converter = DocxToHtmlConverter()

# 转换文件
html_content, messages = converter.convert(temp_file_path)


In [ ]:
from apps.projects.tiptap import docx_to_html

html = docx_to_html(temp_file_path)


In [12]:
import mammoth

def convert_docx_to_html(docx_path):
    with open(temp_file_path, "rb") as docx_file:
        result = mammoth.convert_to_html(docx_file)
        html_content = result.value  # 转换后的 HTML
        messages = result.messages   # 转换过程中可能的警告信息

    return html_content, messages

In [13]:
html, warnings = convert_docx_to_html("example.docx")

In [ ]:
print(html)

In [ ]:
import mammoth

custom_style_map

def convert_docx_to_html(docx_path):
    with open(temp_file_path, "rb") as docx_file:
        result = mammoth.convert_to_html(docx_file)
        html_content = result.value  # 转换后的 HTML
        messages = result.messages   # 转换过程中可能的警告信息

    return html_content, messages

In [16]:
from docx import Document
import mammoth

def get_outline_levels(docx_path):
    """ 解析 DOCX 结构，提取大纲级别 """
    document = Document(docx_path)
    outline_map = {}

    for i, para in enumerate(document.paragraphs):
        if para.style.name.startswith("Heading") or para._element.get("outlineLvl"):
            outline_map[i] = int(para._element.get("outlineLvl", 0))  # 提取大纲级别
            
    return outline_map

In [17]:
outlines = get_outline_levels(temp_file_path)


In [ ]:
print(outlines)

In [ ]:
from apps._tools.docx_parser._01_xml_loader import DocxXMLLoader

loader = DocxXMLLoader(temp_file_path)
from pprint import pprint
pprint(loader.extract_raw())




